In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
# from prophet import Prophet
import pandas as pd
import numpy as np
import seaborn as sns
import lightgbm as lgb
# import catboost as cb
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from tqdm import tqdm
import catboost as cb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import timedelta

pd.set_option('display.max_columns',None)

# MODEL TRAIN

In [ ]:
def add_fts(df):
    columns2shift = df.columns.difference(['day','month','year'])
    for name in columns2shift:
        df[name+"_SHIFT_1"] = df[name].shift(-1).fillna(0)
        df[name+"_SHIFT_2"] = df[name].shift(-2).fillna(0)
        df[name+"_SHIFT_2"] = df[name].shift(-3).fillna(0)
        df[name+"_SHIFT_2"] = df[name].shift(-4).fillna(0)


    for i in range(3,15):
        df[f'tg_shift_{i}'] = df['Цена на арматуру'].shift(i).fillna(0)


In [ ]:
main_train = pd.read_parquet('/content/drive/MyDrive/SAMOLET_CASE/data/train_all_fts.pqt')
main_extra_test = pd.read_parquet('/content/drive/MyDrive/SAMOLET_CASE/data/test_all_fts.pqt')
main_test = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/test.xlsx')
test_real_tg = main_test['Цена на арматуру']
# test['Цена на арматуру'] = 0
main_test = main_test.set_index('dt')

In [ ]:
#добавление новых признаков
def add_features(df, max_lag, roll_mean):
    data = df.copy()

    for name in ['year', 'month', 'day']:
        data[name] = getattr(data.index, name)

    # for lag in range(1, max_lag + 1):
    #     data[f'lag_{lag}'] = data['Цена на арматуру'].shift(lag)


    for clmn_name in data.columns.difference(['day','month','year','Цена на арматуру']):
        for lag in range(1, max_lag + 1):
            data[f"{clmn_name}_LAG_{lag}"] = data[clmn_name].shift(lag)

    data['rolling_mean'] = data['Цена на арматуру'].shift().rolling(roll_mean).mean()

    data['rolling_mean_shift_3'] = data['Цена на арматуру'].shift(15).rolling(roll_mean).mean()

    data['rolling_mean_shift_5'] = data['Цена на арматуру'].shift(29).rolling(roll_mean).mean()

    return data

In [ ]:
main_test.loc[main_test.index,'MAE'] = 0
mae_df= main_test.loc[main_test.index,'MAE']

In [ ]:
# for i in tqdm(range(0,65,7)):
def generate_fts_and_fit(main_train, main_extra_test, main_test,day, N_WEEKS):
    FIRST_DATE = pd.to_datetime("2022-09-05")
    DATE = FIRST_DATE + timedelta(days=day)

    train = pd.concat([main_train,main_extra_test[main_extra_test.index < DATE]])
    test = main_test[main_test.index >= DATE]
    test_real_tg = test['Цена на арматуру']

    train_idx = train.index
    test_idx = test.index

    print(train.shape)
    print(test.shape)

    train['diff_price_1'] = train['Цена на арматуру'].diff(1).fillna(0)
    train['diff_price_1'] = train['Цена на арматуру'].diff(2).fillna(0)

    df = add_features(pd.concat([train,test.drop('Цена на арматуру', axis=1)]), 20, 5).fillna(0)

    train = df.loc[train_idx]
    test = df.loc[test_idx]

    X_train, y_train = train.drop('Цена на арматуру', axis=1), train['Цена на арматуру']
    X_test, y_test = test.drop('Цена на арматуру', axis=1), test_real_tg

    X_test, y_test = X_test.iloc[:N_WEEKS], y_test.iloc[:N_WEEKS]

    model = cb.CatBoostRegressor(random_seed=42,iterations=10000, learning_rate=0.01, early_stopping_rounds=15,loss_function='RMSE',
                                 task_type='CPU',reg_lambda=0.04)
    model.fit(X_train, y_train, eval_set=(X_test, y_test))

    return model, X_test, y_test

model, X_test, y_test = generate_fts_and_fit(main_train, main_extra_test, main_test,day=0,N_WEEKS=10)

(182, 218)
(28, 2)
0:	learn: 13584.8785687	test: 3353.5292718	best: 3353.5292718 (0)	total: 262ms	remaining: 43m 39s
1:	learn: 13477.5798251	test: 3252.9744364	best: 3252.9744364 (1)	total: 447ms	remaining: 37m 16s
2:	learn: 13362.5109350	test: 3249.1244790	best: 3249.1244790 (2)	total: 633ms	remaining: 35m 10s
3:	learn: 13243.5628906	test: 3244.7390422	best: 3244.7390422 (3)	total: 815ms	remaining: 33m 56s
4:	learn: 13132.5365873	test: 3171.5281531	best: 3171.5281531 (4)	total: 1.01s	remaining: 33m 41s
5:	learn: 13016.8782458	test: 3171.4741515	best: 3171.4741515 (5)	total: 1.2s	remaining: 33m 15s
6:	learn: 12898.5452572	test: 3181.6153630	best: 3171.4741515 (5)	total: 1.4s	remaining: 33m 23s
7:	learn: 12790.4033879	test: 3094.0068945	best: 3094.0068945 (7)	total: 1.59s	remaining: 33m 5s
8:	learn: 12695.6330240	test: 3037.7708880	best: 3037.7708880 (8)	total: 1.78s	remaining: 32m 54s
9:	learn: 12580.6206480	test: 2941.9068927	best: 2941.9068927 (9)	total: 1.97s	remaining: 32m 48s
10:	

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,model.predict(X_test))

494.7859769971299

In [ ]:
X_test.to_csv('/content/drive/MyDrive/SAMOLET_CASE/data/X_test.csv')

In [ ]:
preds = pd.DataFrame(main_test.index).set_index('dt')
preds.loc[y_test.index, 'predicted'] = model.predict(X_test)

In [ ]:
model.save_model('/content/drive/MyDrive/SAMOLET_CASE/model/model.cbm')